## ЛАБОРАТОРНАЯ РАБОТА. Определение функционального состояния по ЭЭГ
### Черникова Арина, P41222

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import irfft, rfft, rfftfreq
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

In [8]:
files = ['1_20131027.mat', '1_20131030.mat', '1_20131107.mat', '2_20140404.mat','2_20140413.mat','2_20140419.mat',
        '3_20140603.mat','3_20140611.mat','3_20140629.mat', '4_20140621.mat','4_20140702.mat','4_20140705.mat',
        '5_20140411.mat','5_20140418.mat','5_20140506.mat','6_20130712.mat','6_20131016.mat','6_20131113.mat',
        '7_20131027.mat','7_20131030.mat','7_20131106.mat','8_20140511.mat','8_20140514.mat','8_20140521.mat',
        '9_20140620.mat','9_20140627.mat','9_20140704.mat', '10_20131130.mat','10_20131204.mat','10_20131211.mat',
        '11_20140618.mat','11_20140625.mat','11_20140630.mat', '12_20131127.mat','12_20131201.mat','12_20131207.mat',
        '13_20140527.mat','13_20140603.mat','13_20140610.mat','14_20140601.mat','14_20140615.mat','14_20140627.mat',
        '15_20130709.mat','15_20131016.mat','15_20131105.mat'] 
pre = 'SEED_EEG/Preprocessed_EEG/'


result = []
for file in files:
    data = scipy.io.loadmat(pre + file)
    print(file)
    for j in range(1,16):
        try:
            data_np = data[list(data.keys())[j+2]][2] #FP1 - 0, FP2 - 2
        except:
            print(data_np.keys())
            
        ans = []
        ans.append(data_np.mean())
        ans.append(data_np.max())
        ans.append(data_np.min())
        ans.append(data_np.std())
        
        count = 0
        for i in range(1, len(data_np)):
            if (data_np[i-1] >= 0 and data_np[i] < 0) or (data_np[i-1] <= 0 and data_np[i] > 0):
                count += 1
        ans.append(np.log(count))
        
        y = np.abs(rfft(data_np)) # мощность волн определённой частоты 
        x = rfftfreq(len(data_np), 1 / 200) # список всех частот 

        alphaX = x.copy()
        alphaY = y.copy()
        betaX = x.copy()
        betaY = y.copy()
        gammaX = x.copy()
        gammaY = y.copy()
        thetaX = x.copy()
        thetaY = y.copy()
        deltaX = x.copy()
        deltaY = y.copy()


        for i in range(len(alphaX)):
            if (alphaX[i] < 8) or (alphaX[i] > 13):
                alphaY[i] = 0

        for i in range(len(betaX)):
            if (betaX[i] < 14) or (betaX[i] > 40):
                betaY[i] = 0

        for i in range(len(gammaX)):
            if gammaX[i] < 40:
                gammaY[i] = 0

        for i in range(len(thetaX)):
            if (thetaX[i] < 4) or (thetaX[i] > 8):
                thetaY[i] = 0

        for i in range(len(deltaX)):
            if deltaX[i] > 4:
                deltaY[i] = 0


        alphaY = [elem for elem in alphaY if elem > 0]
        ans.append(np.sum(alphaY))
        ans.append(np.min(alphaY))
        ans.append(np.max(alphaY))
        ans.append(np.std(alphaY))

        betaY = [elem for elem in betaY if elem > 0]
        ans.append(np.sum(betaY))
        ans.append(np.min(betaY))
        ans.append(np.max(betaY))
        ans.append(np.std(betaY))

        gammaY = [elem for elem in gammaY if elem > 0]
        ans.append(np.sum(gammaY))
        ans.append(np.min(gammaY))
        ans.append(np.max(gammaY))
        ans.append(np.std(gammaY))
        

        thetaY = [elem for elem in thetaY if elem > 0]
        ans.append(np.sum(thetaY))
        ans.append(np.min(thetaY))
        ans.append(np.max(thetaY))
        ans.append(np.std(thetaY))

        deltaY = [elem for elem in deltaY if elem > 0]
        ans.append(np.sum(deltaY))
        ans.append(np.min(deltaY))
        ans.append(np.max(deltaY))
        ans.append(np.std(deltaY))

        ans.append(np.sum(alphaY)/np.sum(betaY))
        ans.append(np.sum(alphaY)/np.sum(gammaY))
        ans.append(np.sum(alphaY)/np.sum(thetaY))
        ans.append(np.sum(alphaY)/np.sum(deltaY))
        ans.append(np.sum(betaY)/np.sum(gammaY))
        ans.append(np.sum(betaY)/np.sum(thetaY))
        ans.append(np.sum(betaY)/np.sum(deltaY))
        ans.append(np.sum(gammaY)/np.sum(thetaY))
        ans.append(np.sum(gammaY)/np.sum(deltaY))
        ans.append(np.sum(thetaY)/np.sum(deltaY))    
        
        result.append(ans)
        if j == 7:
            print('half')


1_20131027.mat
half
1_20131030.mat
half
1_20131107.mat
half
2_20140404.mat
half
2_20140413.mat
half
2_20140419.mat
half
3_20140603.mat
half
3_20140611.mat
half
3_20140629.mat
half
4_20140621.mat
half
4_20140702.mat
half
4_20140705.mat
half
5_20140411.mat
half
5_20140418.mat
half
5_20140506.mat
half
6_20130712.mat
half
6_20131016.mat
half
6_20131113.mat
half
7_20131027.mat
half
7_20131030.mat
half
7_20131106.mat
half
8_20140511.mat
half
8_20140514.mat
half
8_20140521.mat
half
9_20140620.mat
half
9_20140627.mat
half
9_20140704.mat
half
10_20131130.mat
half
10_20131204.mat
half
10_20131211.mat
half
11_20140618.mat
half
11_20140625.mat
half
11_20140630.mat
half
12_20131127.mat
half
12_20131201.mat
half
12_20131207.mat
half
13_20140527.mat
half
13_20140603.mat
half
13_20140610.mat
half
14_20140601.mat
half
14_20140615.mat
half
14_20140627.mat
half
15_20130709.mat
half
15_20131016.mat
half
15_20131105.mat
half


In [7]:
data_fp1 = pd.DataFrame(result, columns=['mean','max','min','std','log_via_zero','sum_alpha','min_a','max_a','std_a',
                                        'sum_beta','min_b','max_b','std_b', 'sum_gamma','min_g','max_g','std_g',
                                         'sum_theta','min_t','max_t','std_t','sum_delta','min_d','max_d','std_d',
                                        'a/b','a/g','a/t','a/d','b/g','b/t','b/d','g/t','g/d','t/d'])
data_fp1

,mean,max,min,std,log_via_zero,sum_alpha,min_a,max_a,std_a,sum_beta,...,a/b,a/g,a/t,a/d,b/g,b/t,b/d,g/t,g/d,t/d
0,-0.084421,370.562077,-166.237354,46.274752,7.495542,2.937688e+06,64.998033,9351.746714,1434.891821,6.300681e+06,...,0.466249,0.342135,0.490971,0.093833,0.733803,1.053023,0.201251,1.435021,0.274257,0.191117
1,0.075632,132.888556,-110.745430,30.971143,7.848543,2.191187e+06,21.878554,6901.944047,1044.193014,5.468184e+06,...,0.400716,0.321306,0.592061,0.097801,0.801830,1.477508,0.244067,1.842671,0.304387,0.165188
2,0.021366,168.532133,-135.242939,39.854454,7.526179,2.122831e+06,145.167459,7566.196840,1104.336101,5.434734e+06,...,0.390604,0.304940,0.562725,0.095041,0.780689,1.440653,0.243319,1.845361,0.311672,0.168895
3,0.041841,201.493502,-138.193369,35.700675,7.945910,2.954832e+06,82.853900,9705.196322,1406.443285,6.355995e+06,...,0.464889,0.362022,0.480037,0.105076,0.778729,1.032583,0.226024,1.325986,0.290248,0.218892
4,-0.064285,141.650438,-91.761351,33.062792,7.577634,1.825332e+06,121.080162,6594.742617,1050.373346,4.642100e+06,...,0.393213,0.287501,0.559364,0.105634,0.731159,1.422548,0.268644,1.945607,0.367423,0.188847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,0.509705,301.182270,-142.425299,57.444208,8.025843,4.679730e+06,214.130407,17834.193505,2459.343345,9.395984e+06,...,0.498056,0.416755,0.652032,0.108590,0.836763,1.309153,0.218027,1.564545,0.260560,0.166540
671,0.337664,304.847956,-174.462795,37.120803,9.269364,3.655012e+06,91.977091,11452.727705,1718.303998,1.549671e+07,...,0.235857,0.107270,0.755265,0.142775,0.454807,3.202212,0.605343,7.040813,1.330989,0.189039
672,-0.125619,319.629908,-109.076500,38.916111,8.307953,4.872822e+06,122.952272,18846.695072,2610.606903,1.043513e+07,...,0.466963,0.309738,0.619111,0.162691,0.663302,1.325825,0.348401,1.998824,0.525252,0.262781
673,0.243378,331.670046,-118.851662,36.769359,8.345930,4.690376e+06,59.408242,13934.500714,2314.396331,1.080920e+07,...,0.433924,0.333815,0.542880,0.159818,0.769294,1.251093,0.368308,1.626287,0.478761,0.294389


In [9]:
data_fp2 = pd.DataFrame(result, columns=['mean1','max1','min1','std1','log_via_zero1','sum_alpha1','min_a1','max_a1','std_a1',
                                        'sum_beta1','min_b1','max_b1','std_b1', 'sum_gamma1','min_g1','max_g1','std_g1',
                                         'sum_theta1','min_t1','max_t1','std_t1','sum_delta1','min_d1','max_d1','std_d1',
                                        'a/b1','a/g1','a/t1','a/d1','b/g1','b/t1','b/d1','g/t1','g/d1','t/d1'])
data_fp2

,mean1,max1,min1,std1,log_via_zero1,sum_alpha1,min_a1,max_a1,std_a1,sum_beta1,...,a/b1,a/g1,a/t1,a/d1,b/g1,b/t1,b/d1,g/t1,g/d1,t/d1
0,-0.239872,564.873219,-295.311213,66.722601,8.236421,2.901908e+06,56.192182,8429.632501,1392.395953,9.037161e+06,...,0.321108,0.166626,0.508755,0.079666,0.518909,1.584373,0.248095,3.053276,0.478110,0.156589
1,0.025730,163.137913,-139.594078,30.724153,8.438150,2.195975e+06,32.123602,7541.209012,1025.543120,7.336457e+06,...,0.299324,0.168000,0.641371,0.102015,0.561266,2.142734,0.340820,3.817680,0.607234,0.159058
2,-0.563734,269.949436,-150.412321,46.470729,8.055792,2.153719e+06,22.957064,7083.601078,1097.975392,6.696765e+06,...,0.321606,0.190046,0.596812,0.092028,0.590928,1.855725,0.286153,3.140357,0.484243,0.154200
3,0.049544,382.125378,-215.768814,40.744595,8.407378,2.856437e+06,88.529410,8659.668312,1346.980705,8.086167e+06,...,0.353250,0.204942,0.502663,0.097906,0.580163,1.422967,0.277158,2.452703,0.477725,0.194775
4,-0.024883,180.572271,-155.568123,35.481740,8.214465,1.793372e+06,42.620688,6308.954999,1025.961643,5.845219e+06,...,0.306810,0.173264,0.585398,0.103275,0.564728,1.908015,0.336610,3.378644,0.596058,0.176419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,0.399948,294.446945,-120.729208,53.385018,7.957527,4.696560e+06,152.377663,18699.987839,2552.591116,9.308119e+06,...,0.504566,0.460982,0.680976,0.116397,0.913622,1.349628,0.230686,1.477228,0.252497,0.170926
671,0.214933,276.416540,-168.621540,33.583275,8.819370,3.526395e+06,124.068503,11681.723084,1736.555719,1.131952e+07,...,0.311532,0.211170,0.793535,0.148561,0.677842,2.547199,0.476873,3.757805,0.703516,0.187215
672,-0.172067,294.566154,-107.914209,35.606035,8.384119,4.883879e+06,58.964671,19590.309726,2721.820845,1.044715e+07,...,0.467484,0.371672,0.641163,0.176283,0.795048,1.371519,0.377089,1.725076,0.474297,0.274943
673,0.351376,289.142132,-128.626823,33.427491,8.502486,4.672347e+06,143.686479,12856.340634,2275.026215,1.091110e+07,...,0.428219,0.318436,0.566257,0.174640,0.743629,1.322353,0.407828,1.778244,0.548430,0.308411


In [10]:
data_all = data_fp1.join(data_fp2)
data_all

,mean,max,min,std,log_via_zero,sum_alpha,min_a,max_a,std_a,sum_beta,...,a/b1,a/g1,a/t1,a/d1,b/g1,b/t1,b/d1,g/t1,g/d1,t/d1
0,-0.084421,370.562077,-166.237354,46.274752,7.495542,2.937688e+06,64.998033,9351.746714,1434.891821,6.300681e+06,...,0.321108,0.166626,0.508755,0.079666,0.518909,1.584373,0.248095,3.053276,0.478110,0.156589
1,0.075632,132.888556,-110.745430,30.971143,7.848543,2.191187e+06,21.878554,6901.944047,1044.193014,5.468184e+06,...,0.299324,0.168000,0.641371,0.102015,0.561266,2.142734,0.340820,3.817680,0.607234,0.159058
2,0.021366,168.532133,-135.242939,39.854454,7.526179,2.122831e+06,145.167459,7566.196840,1104.336101,5.434734e+06,...,0.321606,0.190046,0.596812,0.092028,0.590928,1.855725,0.286153,3.140357,0.484243,0.154200
3,0.041841,201.493502,-138.193369,35.700675,7.945910,2.954832e+06,82.853900,9705.196322,1406.443285,6.355995e+06,...,0.353250,0.204942,0.502663,0.097906,0.580163,1.422967,0.277158,2.452703,0.477725,0.194775
4,-0.064285,141.650438,-91.761351,33.062792,7.577634,1.825332e+06,121.080162,6594.742617,1050.373346,4.642100e+06,...,0.306810,0.173264,0.585398,0.103275,0.564728,1.908015,0.336610,3.378644,0.596058,0.176419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,0.509705,301.182270,-142.425299,57.444208,8.025843,4.679730e+06,214.130407,17834.193505,2459.343345,9.395984e+06,...,0.504566,0.460982,0.680976,0.116397,0.913622,1.349628,0.230686,1.477228,0.252497,0.170926
671,0.337664,304.847956,-174.462795,37.120803,9.269364,3.655012e+06,91.977091,11452.727705,1718.303998,1.549671e+07,...,0.311532,0.211170,0.793535,0.148561,0.677842,2.547199,0.476873,3.757805,0.703516,0.187215
672,-0.125619,319.629908,-109.076500,38.916111,8.307953,4.872822e+06,122.952272,18846.695072,2610.606903,1.043513e+07,...,0.467484,0.371672,0.641163,0.176283,0.795048,1.371519,0.377089,1.725076,0.474297,0.274943
673,0.243378,331.670046,-118.851662,36.769359,8.345930,4.690376e+06,59.408242,13934.500714,2314.396331,1.080920e+07,...,0.428219,0.318436,0.566257,0.174640,0.743629,1.322353,0.407828,1.778244,0.548430,0.308411


In [1146]:
data_all.to_csv('fp1_fp2_seed_static_signs.csv')

In [11]:
# data_all = pd.read_csv('static_signs_eeg_seed/fz.csv',index_col=0)
# data_all = data_all.join(pd.read_csv('static_signs_eeg_seed/cz.csv',index_col=0),rsuffix='1')
# data_all

In [12]:
data_all = pd.read_csv('fp1_fp2_seed_static_signs.csv',index_col=0)
data_all

,mean,max,min,std,log_via_zero,sum_alpha,min_a,max_a,std_a,sum_beta,...,a/b1,a/g1,a/t1,a/d1,b/g1,b/t1,b/d1,g/t1,g/d1,t/d1
0,-0.084421,370.562077,-166.237354,46.274752,7.495542,2.937688e+06,64.998033,9351.746714,1434.891821,6.300681e+06,...,0.321108,0.166626,0.508755,0.079666,0.518909,1.584373,0.248095,3.053276,0.478110,0.156589
1,0.075632,132.888556,-110.745430,30.971143,7.848543,2.191187e+06,21.878554,6901.944047,1044.193014,5.468184e+06,...,0.299324,0.168000,0.641371,0.102015,0.561266,2.142734,0.340820,3.817680,0.607234,0.159058
2,0.021366,168.532133,-135.242939,39.854454,7.526179,2.122831e+06,145.167459,7566.196840,1104.336101,5.434734e+06,...,0.321606,0.190046,0.596812,0.092028,0.590928,1.855725,0.286153,3.140357,0.484243,0.154200
3,0.041841,201.493502,-138.193369,35.700675,7.945910,2.954832e+06,82.853900,9705.196322,1406.443285,6.355995e+06,...,0.353250,0.204942,0.502663,0.097906,0.580163,1.422967,0.277158,2.452703,0.477725,0.194775
4,-0.064285,141.650438,-91.761351,33.062792,7.577634,1.825332e+06,121.080162,6594.742617,1050.373346,4.642100e+06,...,0.306810,0.173264,0.585398,0.103275,0.564728,1.908015,0.336610,3.378644,0.596058,0.176419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,0.509705,301.182270,-142.425299,57.444208,8.025843,4.679730e+06,214.130407,17834.193505,2459.343345,9.395984e+06,...,0.504566,0.460982,0.680976,0.116397,0.913622,1.349628,0.230686,1.477228,0.252497,0.170926
671,0.337664,304.847956,-174.462795,37.120803,9.269364,3.655012e+06,91.977091,11452.727705,1718.303998,1.549671e+07,...,0.311532,0.211170,0.793535,0.148561,0.677842,2.547199,0.476873,3.757805,0.703516,0.187215
672,-0.125619,319.629908,-109.076500,38.916111,8.307953,4.872822e+06,122.952272,18846.695072,2610.606903,1.043513e+07,...,0.467484,0.371672,0.641163,0.176283,0.795048,1.371519,0.377089,1.725076,0.474297,0.274943
673,0.243378,331.670046,-118.851662,36.769359,8.345930,4.690376e+06,59.408242,13934.500714,2314.396331,1.080920e+07,...,0.428219,0.318436,0.566257,0.174640,0.743629,1.322353,0.407828,1.778244,0.548430,0.308411


In [13]:
labels = scipy.io.loadmat('SEED_EEG/Preprocessed_EEG/label.mat')['label']
labels = np.array([labels[0]]*45).reshape(1,675)[0]
#np.save('labels_seed',labels)

## Построение моделей

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data_all, labels, test_size=0.25, random_state=249) 

In [16]:
results_table = pd.DataFrame()
results_table['criterion'] = ['accuracy_score', 'f1_score', 'precision_score', 'recall_score']
results_table

,criterion
0,accuracy_score
1,f1_score
2,precision_score
3,recall_score


In [17]:
clf = svm.SVC(kernel='rbf',C=360)
clf.fit(X_train, y_train)
results = clf.predict(X_test)

In [18]:
results_table['svc'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc
0,accuracy_score,0.532544
1,f1_score,0.532035
2,precision_score,0.535456
3,recall_score,0.532544


In [1151]:
# #метод опорных векторов с gridsearch
# parameters_svc = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'),'C':[0.0001, 1000], 'gamma':('scale', 'auto')} #'kernel':('linear', 'rbf'),
# svc = svm.SVC()
# clf = GridSearchCV(svc, parameters_svc, scoring = 'accuracy')
# clf.fit(X_train, y_train)
# results = clf.predict(X_test)
# clf.best_params_

In [1152]:
# results_table['svc_grid'] = [accuracy_score(y_test,results), f1_score(y_test,results), precision_score(y_test,results), recall_score(y_test,results)]
# results_table

In [19]:
tree = DecisionTreeClassifier(criterion='entropy',
 max_depth=8,
 min_samples_split=10,
 min_samples_leaf=5)
tree.fit(X_train, y_train)
results = tree.predict(X_test)

In [20]:
results_table['tree'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc,tree
0,accuracy_score,0.532544,0.538462
1,f1_score,0.532035,0.538647
2,precision_score,0.535456,0.568099
3,recall_score,0.532544,0.538462


In [21]:
clf = RandomForestClassifier(criterion='entropy',
 n_estimators=15,
 max_depth=8,
 min_samples_leaf= 10,
 min_samples_split = 20)
clf.fit(X_train, y_train)
results = clf.predict(X_test)

In [22]:
results_table['forest'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc,tree,forest
0,accuracy_score,0.532544,0.538462,0.514793
1,f1_score,0.532035,0.538647,0.516598
2,precision_score,0.535456,0.568099,0.520770
3,recall_score,0.532544,0.538462,0.514793


In [23]:
clf = KNeighborsClassifier(n_neighbors=55,weights='distance',algorithm='auto')
clf.fit(X_train, y_train)
results = clf.predict(X_test)

In [24]:
results_table['knn'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc,tree,forest,knn
0,accuracy_score,0.532544,0.538462,0.514793,0.497041
1,f1_score,0.532035,0.538647,0.516598,0.491047
2,precision_score,0.535456,0.568099,0.520770,0.505304
3,recall_score,0.532544,0.538462,0.514793,0.497041


In [1159]:
# best = 0.52
# best_random= 0
# params = {}
# for i in range(1,500):
#     clf = svm.SVC(kernel='rbf',C=i)
#     clf.fit(X_train, y_train)
#     results = clf.predict(X_test)
#     if accuracy_score(y_test,results) > best:
#         best = accuracy_score(y_test,results)
#         best_random = i
#     if i % 10 == 0:
#         print(i)

In [25]:
data_nb_x_train = MinMaxScaler().fit(X_train).transform(X_train)
data_nb_x_test = MinMaxScaler().fit(X_test).transform(X_test)
clf = MultinomialNB(alpha=1)
clf.fit(data_nb_x_train, y_train)
results = clf.predict(data_nb_x_test)

In [26]:
results_table['nb'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc,tree,forest,knn,nb
0,accuracy_score,0.532544,0.538462,0.514793,0.497041,0.396450
1,f1_score,0.532035,0.538647,0.516598,0.491047,0.336277
2,precision_score,0.535456,0.568099,0.520770,0.505304,0.457312
3,recall_score,0.532544,0.538462,0.514793,0.497041,0.396450


In [28]:
svc = svm.SVC(kernel='rbf',C=350)
clf = BaggingClassifier(base_estimator=svc, n_estimators=205, random_state=0)
clf.fit(X_train, y_train)
results = clf.predict(X_test)

In [29]:
results_table['bagging_svm'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc,tree,forest,knn,nb,bagging_svm
0,accuracy_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959
1,f1_score,0.532035,0.538647,0.516598,0.491047,0.336277,0.503064
2,precision_score,0.535456,0.568099,0.520770,0.505304,0.457312,0.504223
3,recall_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959


In [30]:
tree = DecisionTreeClassifier(criterion='entropy',
 max_depth=8,
 min_samples_split=10,
 min_samples_leaf=5)
clf = BaggingClassifier(base_estimator=tree, n_estimators=205, random_state=0)
clf.fit(X_train, y_train)
results = clf.predict(X_test)

In [31]:
results_table['bagging_tree'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc,tree,forest,knn,nb,bagging_svm,bagging_tree
0,accuracy_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959,0.591716
1,f1_score,0.532035,0.538647,0.516598,0.491047,0.336277,0.503064,0.595524
2,precision_score,0.535456,0.568099,0.520770,0.505304,0.457312,0.504223,0.610706
3,recall_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959,0.591716


In [32]:
forest = RandomForestClassifier(criterion='entropy',
 n_estimators=15,
 max_depth=8,
 min_samples_leaf= 10,
 min_samples_split = 20)
clf = BaggingClassifier(base_estimator=forest, n_estimators=105, random_state=0)
clf.fit(X_train, y_train)
results = clf.predict(X_test)

In [33]:
results_table['bagging_forest'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc,tree,forest,knn,nb,bagging_svm,bagging_tree,bagging_forest
0,accuracy_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959,0.591716,0.520710
1,f1_score,0.532035,0.538647,0.516598,0.491047,0.336277,0.503064,0.595524,0.524118
2,precision_score,0.535456,0.568099,0.520770,0.505304,0.457312,0.504223,0.610706,0.530570
3,recall_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959,0.591716,0.520710


In [34]:
estimators = [
    ('rf', RandomForestClassifier(criterion='entropy',
 n_estimators=15,
 max_depth=8,
 min_samples_leaf= 10,
 min_samples_split = 20)),
    ('dt', DecisionTreeClassifier(criterion='entropy',
 max_depth=8,
 min_samples_split=10,
 min_samples_leaf=5)),
    ('svc', svm.SVC(kernel='rbf',C=1505)),
    ('knn',KNeighborsClassifier(n_neighbors=55,weights='distance',algorithm='auto'))
    ]

clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()) 
clf.fit(X_train, y_train)
results = clf.predict(X_test)

In [35]:
results_table['stacking'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc,tree,forest,knn,nb,bagging_svm,bagging_tree,bagging_forest,stacking
0,accuracy_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959,0.591716,0.520710,0.538462
1,f1_score,0.532035,0.538647,0.516598,0.491047,0.336277,0.503064,0.595524,0.524118,0.538582
2,precision_score,0.535456,0.568099,0.520770,0.505304,0.457312,0.504223,0.610706,0.530570,0.539828
3,recall_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959,0.591716,0.520710,0.538462


In [36]:
tree = DecisionTreeClassifier(criterion='entropy',
 max_depth=8,
 min_samples_split=10,
 min_samples_leaf=5)
clf = AdaBoostClassifier(base_estimator=tree,n_estimators=45, random_state=0)
clf.fit(X_train, y_train)
results = clf.predict(X_test)

In [37]:
results_table['boost_tree'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc,tree,forest,knn,nb,bagging_svm,bagging_tree,bagging_forest,stacking,boost_tree
0,accuracy_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959,0.591716,0.520710,0.538462,0.538462
1,f1_score,0.532035,0.538647,0.516598,0.491047,0.336277,0.503064,0.595524,0.524118,0.538582,0.536558
2,precision_score,0.535456,0.568099,0.520770,0.505304,0.457312,0.504223,0.610706,0.530570,0.539828,0.538960
3,recall_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959,0.591716,0.520710,0.538462,0.538462


In [38]:
tree = RandomForestClassifier(criterion='entropy',
 n_estimators=15,
 max_depth=8,
 min_samples_leaf= 10,
 min_samples_split = 20)
clf = AdaBoostClassifier(base_estimator=tree,n_estimators=10, random_state=0)
clf.fit(X_train, y_train)
results = clf.predict(X_test)

In [39]:
results_table['boost_forest'] = [accuracy_score(y_test,results), f1_score(y_test,results,average='weighted'), precision_score(y_test,results,average='weighted'), recall_score(y_test,results,average='weighted')]
results_table

,criterion,svc,tree,forest,knn,nb,bagging_svm,bagging_tree,bagging_forest,stacking,boost_tree,boost_forest
0,accuracy_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959,0.591716,0.520710,0.538462,0.538462,0.573964
1,f1_score,0.532035,0.538647,0.516598,0.491047,0.336277,0.503064,0.595524,0.524118,0.538582,0.536558,0.574727
2,precision_score,0.535456,0.568099,0.520770,0.505304,0.457312,0.504223,0.610706,0.530570,0.539828,0.538960,0.578237
3,recall_score,0.532544,0.538462,0.514793,0.497041,0.396450,0.502959,0.591716,0.520710,0.538462,0.538462,0.573964


In [38]:
results_table.to_csv('result_fp1_fp2_seed_static.csv')